In [1]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
    ("user", "Could the members of The Police perform lawful arrests?"),
    ("assistant", "what can the members of The Police do?"),
    ("user", "Jan Sindel’s was born in what country?"),
    ("assistant", "what is Jan Sindel’s personal history?"),
    ("user", "{question}"),
])

In [8]:
question_gen = prompt | ChatAnthropic() | StrOutputParser()

In [9]:
question_gen.invoke({"question":"how is langchain different than langsmith?"})

' What are the differences between langchain and langsmith?'

In [11]:
from langchain.retrievers.you import YouRetriever

In [12]:
retriver = YouRetriever(ydc_api_key=)

In [19]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}
{step_back_context}

Original Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [24]:
chain = {
    "normal_context": (lambda x: x['question']) | retriver | (lambda x: x[:3]),
    "step_back_context": question_gen | retriver | (lambda x: x[:3]),
    "question": lambda x: x["question"]
} | response_prompt | ChatAnthropic() | StrOutputParser()

In [25]:
chain.invoke({"question": "Do the anchors on Rede Globo speak Chinese?"})

" Based on the provided context, there is no direct evidence that the anchors on Rede Globo speak Chinese. However, some key points from the context:\n\n- Rede Globo is a major Brazilian television network, owned by Grupo Globo. \n\n- The context documents discuss Rede Globo's programming and history, but do not mention the language abilities of its anchors.\n\n- As a Brazilian television network, it is likely that Portuguese is the main language used by Rede Globo anchors. \n\n- There is no indication in the provided background information that Rede Globo produces Chinese language content or specifically employs anchors that speak Chinese.\n\nSo in summary, while it is possible some Rede Globo anchors may speak Chinese, there is no direct evidence for this based on the context provided. The main language used on the network is likely Portuguese given its Brazilian origins and audience. Unless specified otherwise, it is unlikely its anchors would need to speak Chinese for their regular